In [1]:
import os
import json
from langchain_community.llms import Ollama
from langchain.prompts import ChatPromptTemplate, PromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

In [2]:
items = [
{'title': 'OPEN THREAD:  Just do it.', 'summary': 'OPEN THREAD: Just do it.'},
{'title': 'THIS PARAGRAPH CAPTURES THE ESSENCE OF THE ATLANTIC’S SAD PIECE ON THE WORLD’S BIGGEST CRUISE SHIP:', 'summary': 'THIS PARAGRAPH CAPTURES THE ESSENCE OF THE ATLANTIC&#8217;S SAD PIECE ON THE WORLD&#8217;S BIGGEST CRUISE SHIP: &#8220;A real journalist or a more intrepid conversationalist would have gone up to the couple and asked them to explain the longevity of their marriage vis-à-vis their love of cruising. But instead I head to my mall suite, take [&#8230;]'},
{'title': 'THE NEW SPACE RACE:  China launches first of a new series of Yaogan reconnaissance satellites.', 'summary': 'THE NEW SPACE RACE: China launches first of a new series of Yaogan reconnaissance satellites.'},
{'title': 'WHY IS THE BIDEN ADMINISTRATION TRYING SO HARD TO KEEP HAMAS FROM LOSING?  Biden Calls for Ceasefire', 'summary': 'WHY IS THE BIDEN ADMINISTRATION TRYING SO HARD TO KEEP HAMAS FROM LOSING? Biden Calls for Ceasefire During Phone Call with Netanyahu.'},
{'title': 'BOOK RECOMMENDATION:  So I just finished up The Naval Odyssey of Prof. James Brand, so far as it goe', 'summary': 'BOOK RECOMMENDATION: So I just finished up The Naval Odyssey of Prof. James Brand, so far as it goes to date, and quite enjoyed it. It&#8217;s a World War II series with plenty of action, but its central character is Prof. James Brand, a wunderkind physicist/operations analyst who zips around solving problems with everything from [&#8230;]'},
{'title': 'TUNED MASS DAMPERS FOR THE WIN:  How a steel ball protected Taiwan’s tallest skyscraper in an eart', 'summary': 'TUNED MASS DAMPERS FOR THE WIN: How a steel ball protected Taiwan’s tallest skyscraper in an earthquake.'},
{'title': 'SCIENCE:  Thinking You Look Younger Has A Curious Link to How You Age.', 'summary': 'SCIENCE: Thinking You Look Younger Has A Curious Link to How You Age.'},
{'title': 'ATTEMPT NO LANDINGS THERE:  Jupiter’s Moon Europa Beckons As NASA’s Clipper Prepares for the Unk', 'summary': 'ATTEMPT NO LANDINGS THERE: Jupiter’s Moon Europa Beckons As NASA’s Clipper Prepares for the Unknown.'},
{'title': 'FLASHBACK:  Time for Consequences:  The “Cabal” who bragged about rigging the 2020 election stuck us', 'summary': 'FLASHBACK: Time for Consequences: The &#8220;Cabal&#8221; who bragged about rigging the 2020 election stuck us with an incapable president at a time of crisis. Examples need to be made.'},
{'title': 'MICROBIOME NEWS:  Gut microbes may influence odds for obesity.', 'summary': 'MICROBIOME NEWS: Gut microbes may influence odds for obesity.'},
]

In [3]:
ollama_url = os.getenv('OLLAMA_URL')
#model_name = "tinydolphin"
model_name = "dolphin-mistral"
model = Ollama(base_url=ollama_url, model=model_name, keep_alive=5, temperature=0.0)

In [4]:
parser = StructuredOutputParser.from_response_schemas(
  [ResponseSchema(name="title", description="title of the article")]
)

prompt = PromptTemplate(
	template="{data}\n{format_instructions}\n",
	input_variables=["data"],
	partial_variables={"format_instructions": parser.get_format_instructions()},
)

# format chat prompt
system_prompt = SystemMessage(content=("""
	You are an expert news article title editor.
  Use the provided title and summary to write a concise and accurate title that is concise, informative and avoids sounding like clickbait. 
	Do not include links or urls in the title.
	Title should be as short as possible, aim to be less that 70 characters long.
	Title should have an absolute minimum of punctuation and use at most one all capitalized word at the start of the title.
	"""))
user_prompt = HumanMessagePromptTemplate(prompt=prompt)

chat_prompt = ChatPromptTemplate.from_messages([system_prompt, user_prompt])
chain = chat_prompt | model | parser

In [5]:
data = []
for item in items:
  data.append({"data": json.dumps(item)})

rows = chain.batch(data, max_concurrency=len(data))

index = 1
for row in rows:
  print(f"{index:2d}: {row['title']}")
  index += 1


 1: OPEN THREAD: Just Do It.
 2: World's Biggest Cruise Ship: A Couple's Longevity and Love for Cruising
 3: China Launches First Yaogan Reconnaissance Satellite in New Series
 4: Biden Calls for Ceasefire Amid Hamas-Israel Conflict
 5: World War II Naval Odyssey: A Thrilling Read
 6: Taiwan's Tallest Skyscraper Stands Firm in Earthquake: Tuned Mass Dampers at Work
 7: Thinking You Look Younger Linked to Aging Rate
 8: NASA's Clipper Prepares for Europa Mission
 9: 2020 Election Rigging: Consequences for the Cabal
10: Gut Microbes Linked to Obesity Risk
